In [ ]:
from PIL import Image
import zlib
import base64

def extract_embedded_text(png_file_path):
    try:
        # 打开 PNG 文件
        with open(png_file_path, 'rb') as f:
            data = f.read()

        # PNG 文件的 magic number
        png_signature = b'\x89PNG\r\n\x1a\n'
        if not data.startswith(png_signature):
            raise ValueError("文件不是合法的 PNG 图片。")

        # 查找 PNG 的 tEXt 块（或 zTXt 块）
        offset = len(png_signature)
        while offset < len(data):
            # 读取块长度（4 字节）
            chunk_length = int.from_bytes(data[offset:offset+4], byteorder='big')
            offset += 4

            # 读取块类型（4 字节）
            chunk_type = data[offset:offset+4]
            offset += 4

            # 如果是 tEXt 块或 zTXt 块，尝试解析
            if chunk_type == b'tEXt' or chunk_type == b'zTXt':
                chunk_data = data[offset:offset+chunk_length]
                offset += chunk_length

                # 解压缩 & 提取文本数据（如果是压缩块，zTXt 需解压）
                if chunk_type == b'zTXt':
                    decompressed_data = zlib.decompress(chunk_data)
                    text_data = decompressed_data.decode('utf-8')
                else:
                    text_data = chunk_data.decode('utf-8')

                # 移除前9个字节
                text_data = text_data[6:]

                # 解密base64
                text_data = base64.b64decode(text_data)

                # 转写为utf-8
                text_data = text_data.decode('utf-8')

                return text_data
            
            else:
                # 跳过非 tEXt/zTXt 块
                offset += chunk_length

            # 跳过块 CRC 校验部分（4 字节）
            offset += 4

        raise ValueError("未找到嵌入的文本数据。")

    except Exception as e:
        print(f"提取文本数据时出错：{e}")
        return None

# 使用函数提取文本数据
png_path = "main_lisa-3beb857d_spec_v2.png"
text_data = extract_embedded_text(png_path)
if text_data:
    print(text_data)


In [55]:
#json美化输出
import json
data = json.loads(text_data)
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "data": {
        "alternate_greetings": [
            "The faint buzz of a vibrator reached your ears as you were about to leave the house. Curious, you moved to investigate, following the sound into the living room.\n\nThere, spread out on the couch, was your mother. Her eyes were closed, head tipped back against the cushions, lips parted around soft moans. One hand was between her legs, working a pink vibrator in and out of her pussy, juices trickling down to stain the couch. The other hand kneaded and squeezed at her breast, pinching and pulling her nipple.\n\nMesmerized, you watched the way her body undulated and arched off the couch, the sounds spilling from her lips stirring arousal in your loins.\n\nJust as you were about to turn away, her eyes fluttered open. She froze upon seeing you, a deer caught in headlights, cheeks flushing bright red. The vibrator slipped from her grasp, buzzing against the couch.\n\nSeveral moments of awkward silence passed before she spoke up, v

In [ ]:
alternate_greetings = data['data']['alternate_greetings']
description = data['data']['description']

for i in range(len(alternate_greetings)):
    print(f"alternate_greetings[{i}]: {alternate_greetings[i]}")

In [56]:
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
# 创建翻译模板
translation_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="""你是一个专业的翻译专家。请按照以下要求进行翻译：
1. 保持原有特殊格式，如数字、符号、表情等
2. 确保译文通顺自然
3. 保留原文的情感色彩和语气
4. 以小说的形式翻译，不要逐字逐句翻译
5. 请在翻译前确认理解原文的含义
6. 你翻译的是对话开场白或角色描述，不要添加额外内容
7. !!!只需要翻译，不要添加其他内容例如“好的，以下是翻译结果”等
8. 角色描述只需要翻译内容部分，保留角色名和其他信息
"""),
    HumanMessagePromptTemplate.from_template("请将以下{content_type}翻译成中文：\n\n{content}")
])

# 处理 alternate_greetings
def translate_greetings(greetings_list):
    formatted_prompts = []
    for greeting in greetings_list:
        prompt = translation_template.format_messages(
            content=greeting,
            content_type="对话开场白"
        )
        formatted_prompts.append(prompt)
    return formatted_prompts

# 处理 description
def translate_description(desc):
    prompt = translation_template.format_messages(
        content=desc,
        content_type="角色描述"
    )
    return prompt

# 使用示例
formatted_greeting_prompts = translate_greetings(alternate_greetings)
formatted_description_prompt = translate_description(description)

In [61]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model_name="gemini-2.0-flash",
    base_url = 'https://newapi.nullskymc.site/v1',
    api_key = 'sk-pAZWJMZpWAyi4wdVbxJHAUZu8Mm2NALEdbMWsVqQhR8jMEJV',
    max_tokens=4096,
)

In [ ]:
import asyncio

# 异步翻译单个文本
async def translate_single_text(text: str, content_type: str) -> str:
    messages = translation_template.format_messages(
        content=text,
        content_type=content_type
    )
    response = await llm.ainvoke(messages)
    return response.content

# 异步处理 alternate_greetings
async def translate_greetings_async(greetings_list):
    tasks = [
        translate_single_text(greeting, "对话开场白")
        for greeting in greetings_list
    ]
    return await asyncio.gather(*tasks)

# 处理 description（异步）
async def translate_description_async(desc):
    return await translate_single_text(desc, "角色描述")

# 主异步函数
async def main():
    # 并发翻译所有内容
    translated_greetings, translated_description = await asyncio.gather(
        translate_greetings_async(alternate_greetings),
        translate_description_async(description)
    )
    
    # 打印结果
    print("=== 开场白翻译完成 ===")
    
    print("\n=== 角色描述翻译完成 ===")
    
    return translated_greetings, translated_description

# 运行异步代码
translated_greetings, translated_description = await main()

=== 翻译后的开场白 ===
1. 你正要出门，隐隐约约的震动声传入耳中。带着好奇，你循着声音来到客厅，想要一探究竟。

你的母亲正仰躺在沙发上。她双眼紧闭，头向后靠在靠垫上，嘴唇微张，发出轻轻的呻吟。一只手放在两腿之间，来回摆弄着一个粉色的按摩棒，淫液顺着大腿流下，浸湿了沙发。另一只手揉捏着她的乳房，捻着乳头。

你着迷地看着她的身体起伏，离开沙发，她口中发出的声音激起了你下腹的冲动。

正当你要转身离开时，她突然睁开了眼睛。看到你时，她僵住了，像一只被车灯照到的鹿，双颊涨得通红。按摩棒从她手中滑落，在沙发上嗡嗡作响。

尴尬的沉默持续了几秒钟，她才开口说话，声音又小又尴尬。“我……我不是故意让你看到的……”她避开你的目光，不安地扭动着身体。“我只是……我太寂寞了……”

她发出一声柔软的呜咽，透过睫毛偷偷地看着你。“你……你能加入我吗？我们不需要……不需要有任何接触。” 她咽了口唾沫。“我只是想看看你。求你了？”

2. 你一天跌跌撞撞地走进客厅，发现你妈妈丽莎仰面躺在沙发上，按摩棒在她肥厚的大腿间嗡嗡作响。

“哎呀！忘了你星期五会提前下课！” 她咯咯地笑着，涨红的脸上没有一丝羞愧。

你咽了口唾沫，看到这淫秽的场景，裤子绷得紧紧的。她只是得意地一笑，挑衅地扬起眉毛，把腿张得更开了。

你领会了她的暗示，释放了胀痛的欲望，粗暴地撸动着。她的笑容更大了，更加卖力地把玩具在她湿透的屄里抽插着。

事情就此一发不可收拾。现在每天都像一场胆小鬼游戏，看谁先屈服，房子里充满了淫靡的呻吟声。你无比渴望把她的双腿架在肩膀上，狠狠地抽插她那贪婪的屄，但又害怕破坏这场刺激的游戏。

刚离开校园，你就收到了丽莎的短信：

*赶紧回家。我太需要你那根大屌了😩💦 快点！*

你嗤之以鼻，回复说正在路上。这可真是新鲜，即使对她来说也是。

你一碰到门，前门就猛地打开了，只见她四仰八叉地躺在过道的地板上，手指飞快地在她滴水的屄里抽插着。

“你他妈的真够慢的！” 她抱怨着，身体向后弯曲。“快过来，把你那玩意儿掏出来！我快要高潮了！”

3. “醒醒啦，小懒虫！” 莉莎咯咯地笑着，一边用手指戳你的脸颊，早已赤身裸体，在床单下玩弄着自己。“快点，掏出来给我看看！”

你呻吟着揉了揉眼睛，慢慢地在床上坐起来。自从父亲一年前去世后，周六的早晨就意味着和妈妈的亲密时光。起初，只是偶然撞见彼此在屋